# Pré-processamento Pilar de Estampa

Finalmente, nos resta identificar um subespaço do embedding que seja capaz de capturar a similaridade entre estampas. Para isso, vamos usar os atributos tagueados pelo gpt: ESTAMPA (tipo de estampa, como floral, listrado, etc), ZOOM_ESTAMPA (baixo, medio ou alto zoom) e LOCALIZACAO_ESTAMPA (se a estampa é localizada ou se espalha por toda a peça).

Nesse notebook vamos analisar a distribuição desses atributos e como eles se relacionam com a similaridade entre estampas.

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [28]:
# Importando base categorizada pelo gpt
df_base_categorizada = pd.read_parquet('datasets/base_animale_categorizada_4o.parquet')

In [29]:
df_base_categorizada.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 766 entries, 0 to 765
Data columns (total 28 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   file_path            766 non-null    object
 1   id_produto           766 non-null    object
 2   grupo_produto        766 non-null    object
 3   file_name            766 non-null    object
 4   original_file_path   766 non-null    object
 5   link                 766 non-null    object
 6   PRODUTOS             765 non-null    object
 7   CARACTERISTICAS      765 non-null    object
 8   ESTAMPA              764 non-null    object
 9   AMARRACAO_PESCOCO    215 non-null    object
 10  COMPRIMENTO_BLAZER   18 non-null     object
 11  COR_PREDOMINANTE     764 non-null    object
 12  COMPRIMENTO_SHORT    27 non-null     object
 13  MODELAGEM_SHORT      27 non-null     object
 14  DECOTE_OU_GOLA       514 non-null    object
 15  AMARRACAO_CORPO      746 non-null    object
 16  ALTURA_C

In [30]:
df_base_categorizada['ESTAMPA'].value_counts()

# Parece ruim. Devo olhar pro cadastro de estampas e fazer um regex?

SEM_ESTAMPA            546
GEOMETRICO              48
FLORAL                  40
ANIMALPRINT_ONCA        30
LISTRADO_VERTICAL       22
ABSTRATO                18
LISTRADO_HORIZONTAL     16
MINIMALISTA             12
ANIMALPRINT_COBRA       10
POA                      7
FOLHAGEM                 4
XADREZ                   3
LISTRADO_OUTRO           2
OUTRA                    1
TROPICAL                 1
TIE_DYE                  1
DEGRADE                  1
ESTAMPA_ETNICA           1
MARMORIZADA              1
Name: ESTAMPA, dtype: int64

In [31]:
df_base_categorizada['ZOOM_ESTAMPA'].value_counts()

N/A        539
MEDIO      121
PEQUENO     74
GRANDE      30
Name: ZOOM_ESTAMPA, dtype: int64

In [32]:
df_base_categorizada['LOCALIZACAO_ESTAMPA'].value_counts()

# DISPENSAVEL?

N/A              538
ROUPA_INTEIRA    226
Name: LOCALIZACAO_ESTAMPA, dtype: int64

Puxando descrição da cor da base inicial

In [33]:
base = pd.read_parquet('dataset_images/base_animale.parquet')

In [34]:
base.sample(3)

,shelf,href,imgs,descricao,preco,grupo,link 1° imagem,link produto,id,codigo,cor
16,VERÃO 25,/vestido-de-seda-curto-frente-unica-estampado-...,[https://lojaanimale.vtexassets.com/arquivos/i...,vestido-de-seda-curto-frente-unica-estampado,"R$ 1.698,00",vestido,https://lojaanimale.vtexassets.com/arquivos/id...,https://www.animale.com.br/vestido-de-seda-cur...,07-20-6833-09812,320140/p,est-loc-abstrato-solsticio-p
611,VERÃO 25,/calca-cenoura-clochard-com-faixa-de-algodao-p...,[https://lojaanimale.vtexassets.com/arquivos/i...,calca-cenoura-clochard-com-faixa-de-algodao-preta,"R$ 598,00",calca,https://lojaanimale.vtexassets.com/arquivos/id...,https://www.animale.com.br/calca-cenoura-cloch...,25-34-1917-0005,317537/p,preto
497,VERÃO 25,/blusa-de-georgete-sem-alcas-franzido-bege-man...,[https://lojaanimale.vtexassets.com/arquivos/i...,blusa-de-georgete-sem-alcas-franzido,"R$ 998,00",top-blusa,https://lojaanimale.vtexassets.com/arquivos/id...,https://www.animale.com.br/blusa-de-georgete-s...,52-13-5171-09041,304264/p,bege-manha


In [35]:
# Filtrar base para apenas id's que estão em df_base_categorizada
base = base[base['id'].isin(df_base_categorizada['id_produto'])]

In [36]:
base.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 766 entries, 0 to 891
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   shelf           766 non-null    object
 1   href            766 non-null    object
 2   imgs            766 non-null    object
 3   descricao       766 non-null    object
 4   preco           766 non-null    object
 5   grupo           766 non-null    object
 6   link 1° imagem  766 non-null    object
 7   link produto    766 non-null    object
 8   id              766 non-null    object
 9   codigo          766 non-null    object
 10  cor             766 non-null    object
dtypes: object(11)
memory usage: 71.8+ KB


In [37]:
# Criando coluna que verifica com regex se a coluna cor contém a palavra 'est'
base['ESTAMPADO'] = base['cor'].str.contains('est', case=False, na=False)

In [38]:
base['ESTAMPADO'].value_counts()

False    640
True     126
Name: ESTAMPADO, dtype: int64

### Ok, talvez seja melhor confiar no cadastro

abstrato, onca, floral, cobra, grafico, aquarela, listra, pois(poa), xadrez, folhagem, leopardo, geo(geometrico), liberty camomila (floral?), gravataria

In [39]:
base.loc[base['ESTAMPADO'] == True, 'cor'].value_counts()

est-pois-tri-preto-e-exotique               6
est-gravataria-flor-preto-e-kirli           4
est-geo-caminho                             4
est-floral-cura-aquarius-m                  4
est-onca-aquarela-biscoito                  4
est-onca-moderna-color                      4
est-cobra-biscione-marzipan                 4
est-cobra-biscione-natureza                 4
est-liberty-camomila-preto-p                4
est-onca-nite-strawberry-p                  3
est-onca-nite-color-p                       3
est-abstrato-silhuetas-yucca                3
est-floral-geometrico-block-color           3
est-floral-leve-narciso                     3
est-onca-elixir-color                       3
est-pois-ciclos-tofu-e-preto                3
est-onca-aquarela-forte-color-p             3
est-floral-pastel-preto-e-leve              3
est-onca-leve-flan                          3
est-floral-cura-color-m                     3
est-pois-passaro-marinho-p                  3
est-pois-grafico-color            

In [ ]:
# possiveis estampas: abstrato, onca, floral, cobra, grafico, aquarela, listra, pois, xadrez, folhagem, leopardo, geo 

dict_estampas = {
    'abstrato|peixes': 'abstrato',
    'onca': 'onca',
    'floral|camomila': 'floral',
    'cobra': 'cobra',
    # 'grafic': 'grafico',
    'aquarela': 'aquarela',
    'listra': 'listra',
    'pois': 'poa',
    'xadrez': 'xadrez',
    'folhagem': 'folhagem',
    'leopardo': 'leopardo',
    'geo|gravataria': 'geometrico'
}


# Inicializando a coluna como None
base['ESTAMPA_CADASTRO'] = None

# Preenchendo com a estampa correspondente
for estampa, valor in dict_estampas.items():
    # Preenche apenas onde não há valor ainda e há correspondência
    mask = base['cor'].str.contains(estampa, case=False, na=False) & base['ESTAMPA_CADASTRO'].isna() & base['ESTAMPADO']
    base.loc[mask, 'ESTAMPA_CADASTRO'] = valor


In [41]:
base.sample(5)

,shelf,href,imgs,descricao,preco,grupo,link 1° imagem,link produto,id,codigo,cor,ESTAMPADO,ESTAMPA_CADASTRO
583,VERÃO 25,/calca-com-cos-transpassado-bege-gergelim-bege...,[https://lojaanimale.vtexassets.com/arquivos/i...,calca-com-cos-transpassado-bege-gergelim,"R$ 1.198,00",calca,https://lojaanimale.vtexassets.com/arquivos/id...,https://www.animale.com.br/calca-com-cos-trans...,25-34-1600-09247,312995/p,bege-gergelim,False,None
454,VERÃO 25,/blusa-decote-v-e-manga-longa-raglan-de-malha-...,[https://lojaanimale.vtexassets.com/arquivos/i...,blusa-decote-v-e-manga-longa-raglan-de-malha,"R$ 228,00",top-blusa,https://lojaanimale.vtexassets.com/arquivos/id...,https://www.animale.com.br/blusa-decote-v-e-ma...,52-13-5461-09253,315755/p,verde-encanto,False,None
452,VERÃO 25,/blusa-de-georgette-verde-com-tunel-no-decote-...,[https://lojaanimale.vtexassets.com/arquivos/i...,blusa-de-georgette-verde-com-tunel-no-decote,"R$ 558,00",top-blusa,https://lojaanimale.vtexassets.com/arquivos/id...,https://www.animale.com.br/blusa-de-georgette-...,52-13-5394-09253,312165/p,verde-encanto,False,None
499,VERÃO 25,/blusa-de-tricot-com-textura-preta-manga-longa...,[https://lojaanimale.vtexassets.com/arquivos/i...,blusa-de-tricot-com-textura-preta-manga-longa,"R$ 698,00",top-blusa,https://lojaanimale.vtexassets.com/arquivos/id...,https://www.animale.com.br/blusa-de-tricot-com...,52-13-4821-0005,304735/p,preto,False,None
730,VERÃO 25,/mini-saia-em-estampa-cobra-verde-25-11-1173-0...,[https://lojaanimale.vtexassets.com/arquivos/i...,mini-saia-em-estampa-cobra,"R$ 598,00",saia,https://lojaanimale.vtexassets.com/arquivos/id...,https://www.animale.com.br/mini-saia-em-estamp...,25-11-1173-0300,308520/p,verde,False,None


In [42]:
base.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 766 entries, 0 to 891
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   shelf             766 non-null    object
 1   href              766 non-null    object
 2   imgs              766 non-null    object
 3   descricao         766 non-null    object
 4   preco             766 non-null    object
 5   grupo             766 non-null    object
 6   link 1° imagem    766 non-null    object
 7   link produto      766 non-null    object
 8   id                766 non-null    object
 9   codigo            766 non-null    object
 10  cor               766 non-null    object
 11  ESTAMPADO         766 non-null    bool  
 12  ESTAMPA_CADASTRO  126 non-null    object
dtypes: bool(1), object(12)
memory usage: 78.5+ KB


In [43]:
base['ESTAMPADO'].value_counts()

False    640
True     126
Name: ESTAMPADO, dtype: int64

In [44]:
base['ESTAMPA_CADASTRO'].value_counts()

floral        31
onca          27
poa           21
geometrico    14
listra         9
cobra          8
abstrato       7
xadrez         5
leopardo       2
folhagem       2
Name: ESTAMPA_CADASTRO, dtype: int64

In [45]:
# Quem que é estampado e não tem estampa cadastrada?
base.loc[(base['ESTAMPADO'] == True) & (base['ESTAMPA_CADASTRO'].isna()), 'cor'].value_counts()

Series([], Name: cor, dtype: int64)

Temos as estampas dos 165 que são estampados, o resto sabemos que é liso.

Agora, nossa tarefa envolve:
- 1. Classificar se é ESTAMPADO ou LISO
- 2. Se for estampado, classificar o tipo de estampa (floral, listrado, etc)

In [46]:
base.columns

Index(['shelf', 'href', 'imgs', 'descricao', 'preco', 'grupo',
       'link 1° imagem', 'link produto', 'id', 'codigo', 'cor', 'ESTAMPADO',
       'ESTAMPA_CADASTRO'],
      dtype='object')

In [47]:
base.sample(3)

,shelf,href,imgs,descricao,preco,grupo,link 1° imagem,link produto,id,codigo,cor,ESTAMPADO,ESTAMPA_CADASTRO
158,VERÃO 25,/vestido-mini-sem-alca-garden-bege-whitecap-07...,[https://lojaanimale.vtexassets.com/arquivos/i...,vestido-mini-sem-alca-garden,"R$ 1.398,00",vestido,https://lojaanimale.vtexassets.com/arquivos/id...,https://www.animale.com.br/vestido-mini-sem-al...,07-20-6379-04346,309196/p,bege-whitecap,False,None
814,VERÃO 25,/short-de-couro-curto-pregas-preto-25-05-3986-...,[https://lojaanimale.vtexassets.com/arquivos/i...,short-de-couro-curto-pregas,"R$ 3.798,00",short,https://lojaanimale.vtexassets.com/arquivos/id...,https://www.animale.com.br/short-de-couro-curt...,25-05-3986-0005,301184/p,preto,False,None
644,VERÃO 25,/calca-de-couro-cos-abaulado-lilas-nuvem-25-34...,[https://lojaanimale.vtexassets.com/arquivos/i...,calca-de-couro-cos-abaulado,"R$ 5.298,00",calca,https://lojaanimale.vtexassets.com/arquivos/id...,https://www.animale.com.br/calca-de-couro-cos-...,25-34-1442-09040,301259/p,lilas-nuvem,False,None


In [48]:
df_estampa = base[['id', 'descricao', 'cor', 'ESTAMPADO', 'ESTAMPA_CADASTRO']]

In [49]:
df_estampa.sample(5)

,id,descricao,cor,ESTAMPADO,ESTAMPA_CADASTRO
586,25-34-1654-0005,calca-reta-preta-com-pregas,preto,False,None
577,25-34-1720-0005,calca-de-georgette-pesado-ampla-preta-cos-com-...,preto,False,None
27,07-20-6778-0005,vestido-tricot-transparencia,preto,False,None
98,07-20-6547-09621,vestido-de-georgette-longo-com-pregas-estampado,est-onca-moderna-color,True,onca
203,07-20-6200-0005,vestido-de-couro-midi-alca-recortes,preto,False,None


In [50]:
df_estampa.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 766 entries, 0 to 891
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                766 non-null    object
 1   descricao         766 non-null    object
 2   cor               766 non-null    object
 3   ESTAMPADO         766 non-null    bool  
 4   ESTAMPA_CADASTRO  126 non-null    object
dtypes: bool(1), object(4)
memory usage: 30.7+ KB


In [52]:
df_estampa['ESTAMPA_CADASTRO'].value_counts()

floral        31
onca          27
poa           21
geometrico    14
listra         9
cobra          8
abstrato       7
xadrez         5
leopardo       2
folhagem       2
Name: ESTAMPA_CADASTRO, dtype: int64

In [51]:
# Salvando em datasets/PILAR_ESTAMPA
df_estampa.to_parquet('datasets/PILAR_ESTAMPA/df_estampa.parquet')